### Importing Relevant Packages

In [ ]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Loading Data and Preprocessing

In [ ]:
 raw_data = yfinance.download(tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = '1994-01-07', end = '2021-04-13', 
                              interval = '1d', group_by = 'ticker', auto_adjust = True, treads = True)

In [ ]:
df_comp = raw_data.copy()

In [ ]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

### Removing unwanted Columns

In [ ]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']

### Setting Frequency and Filling Missing Values

In [ ]:
df_comp = df_comp.asfreq('b')
df_comp = df_comp.fillna(method = 'ffill')

### Creating Returns

In [ ]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1).mul(100)
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1).mul(100)
df_comp['ret_dax'] = df_comp.dax.pct_change(1).mul(100)
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1).mul(100)

### Normalizing

In [ ]:
#Setting the benchmark
bench_1 = df_comp['ret_spx'].iloc[1]
bench_2 = df_comp['ret_ftse'].iloc[1]
bench_3 = df_comp['ret_dax'].iloc[1]
bench_4 = df_comp['ret_nikkei'].iloc[1]

In [ ]:
#Normailzing the Returns
df_comp['norm_ret_spx'] = df_comp.ret_spx.div(bench_1).mul(100)
df_comp['norm_ret_ftse'] = df_comp.ret_ftse.div(bench_2).mul(100)
df_comp['norm_ret_dax'] = df_comp.ret_dax.div(bench_3).mul(100)
df_comp['norm_ret_nikkei'] = df_comp.ret_nikkei.div(bench_4).mul(100)

### Splitting the Data

In [ ]:
size = int(len(df_comp) * 0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [ ]:
df.tail()

In [ ]:
df_test.tail()

### Fitting

In [ ]:
start_date = '2015-10-28'
end_date = '2021-04-12'
model_auto = auto_arima(df.ret_ftse[1:])
df_auto_pred = pd.DataFrame(model_auto.predict(n_periods = len(df_test[start_date:end_date])), 
                           index = df_test[start_date:end_date].index)

In [ ]:
model_auto.summary()

In [ ]:
df_auto_pred.plot(figsize = (20,5), color = 'red')
df_test.ret_ftse[start_date:end_date].plot(color = 'blue')
plt.title('Auto Model Predictions vs Real Data', size = 24)
plt.show()

### Auto Model (Complex)

In [ ]:
model_auto_comp = auto_arima(df.ret_ftse[1:], exogenous = df[['ret_spx', 'ret_dax', 'ret_nikkei']][1:],
                            m = 5, max_p = 5, max_q = 5, max_P = 5, max_Q = 5)
df_auto_comp_pred = pd.DataFrame(model_auto_comp.auto(n_periods = len(df_test[start_date:end_date]), 
                                exogenous =  df_test[['ret_spx','ret_dax','ret_nikkei']][start_date:end_date]),
                                index = df_test[start_date:end_date].index)

In [ ]:
df_auto_comp_pred.plot(figsize = (20,5), color = 'red')
df_test.ret_ftse[start_date:end_date].plot(color = 'blue')
plt.title('Auto Model Predictions vs Real Data', size = 24)
plt.show()